In [22]:
# 旧代码存在一个错误；气候类型应该减小颗粒度；编码方式不合理
onehot = {'EF': 0, 'ET': 1,
                    'Af': 2, 'Am': 3,
                    'Aw': 4,
                    'As': 5,
                    'BWh': 6,
                    'BWk': 7,
                    'BSh': 8,
                    'BSk': 9,
                    'Csa': 10,
                    'Csb': 11,
                    'Csc': 12,
                    'Cwa': 13,
                    'Cfa': 14,
                    'Cfb': 15,
                    'Cfc': 16,
                    'Cwb': 17,
                    'Cwc': 18,
                    'Dfa': 19,
                    'Dfb': 20,
                    'Dwa': 21,
                    'Dwb': 22,
                    'Dsa': 23,
                    'Dsb': 24,
                    'Dfc': 25,
                    'Dfd': 26,
                    'Dwc': 27,
                    'Dwd': 28,
                    'Dsc': 29,
                    'Dsd': 30,
                    'undefined': 31}
def koppen(monthlyTemperature, monthlyPrecipitation, hemisphere):
    monthlyTemperatureSorted = sorted(monthlyTemperature)
    monthlyPrecipitationSorted = sorted(monthlyTemperature)
    totalPrecipitation = sum(monthlyPrecipitation)
    precipitationIntermediate = 100 - totalPrecipitation / 25
    # E Category
    if (monthlyTemperatureSorted[11] < 0):
        return 'EF'
    if (monthlyTemperatureSorted[11] < 10 and monthlyTemperatureSorted[11] >= 0):
        return 'ET'
    # A Category
    if (monthlyTemperatureSorted[0] >= 18):
        if (monthlyPrecipitationSorted[0] >= 60):
            return 'Af'
        if (monthlyPrecipitationSorted[0] < 60 
            and monthlyPrecipitationSorted[0] >= precipitationIntermediate):
            return 'Am'
        # As / Aw
        driestMonth = monthlyPrecipitation.index(min(monthlyPrecipitation))
        # April - September: North Hemisphere
        if ('N' in hemisphere):
            if (driestMonth >= 3 and driestMonth <= 8):
                return 'As'
            else:
                return 'Aw'
        if ('S' in hemisphere):
            if (driestMonth >= 3 and driestMonth <= 8):
                return 'Aw'
            else:
                return 'As'
    # K Value
    # summerPrecipitation = Precipitation of April - September
    summerPrecipitation = sum(monthlyPrecipitation[3:9])
    if ('S' in hemisphere):
        summerPrecipitation = totalPrecipitation - summerPrecipitation
    K = sum(monthlyTemperature) / 12 * 20
    if (summerPrecipitation >= totalPrecipitation * 0.7):
        K = K + 280
    elif (summerPrecipitation >= totalPrecipitation * 0.3):
        K = K + 140
    # B Category
    # BW
    if (totalPrecipitation < K * 0.5):
        if (sum(monthlyTemperature) >= 216 and monthlyTemperatureSorted[0] < 18):
            return 'BWh'
        if (sum(monthlyTemperature) < 216):
            return 'BWk'
    # BS
    if (totalPrecipitation >= K * 0.5 and totalPrecipitation < K):
        if (sum(monthlyTemperature) >= 216 and monthlyTemperatureSorted[0] < 18):
            return 'BSh'
        if (sum(monthlyTemperature) < 216):
            return 'BSk'
    # C,D Category
    # winter / summer Humidest / Driest Precipitation
    if ('N' in hemisphere):
        winterHumidestPrecipitation = max(max(monthlyPrecipitation[0:3]), max(monthlyPrecipitation[9:12]))
        winterDriestPrecipitation = min(min(monthlyPrecipitation[0:3]), min(monthlyPrecipitation[9:12]))
        summerHumidestPrecipitation = max(monthlyPrecipitation[3:9])
        summerDriestPrecipitation = min(monthlyPrecipitation[3:9])
    if ('S' in hemisphere):
        winterHumidestPrecipitation = max(monthlyPrecipitation[3:9])
        winterDriestPrecipitation = min(monthlyPrecipitation[3:9])
        summerHumidestPrecipitation = max(max(monthlyPrecipitation[0:3]), max(monthlyPrecipitation[9:12]))
        summerDriestPrecipitation = min(min(monthlyPrecipitation[0:3]), min(monthlyPrecipitation[9:12]))
    # C / D
    if (totalPrecipitation >= K and monthlyTemperatureSorted[11] >= 10):
        if (monthlyTemperatureSorted[0] >= 0 and monthlyTemperatureSorted[0] < 18):
            result = 'C'
        if (monthlyTemperatureSorted[0] < 0):
            result = 'D'
        # s / w / f
        if (winterHumidestPrecipitation >= 3 * summerDriestPrecipitation): 
            result = result + 's'
        elif (summerHumidestPrecipitation >= 10 * winterDriestPrecipitation): 
            result = result + 'w'
        else:
            result = result + 'f'
        # a / b / c
        if (monthlyTemperatureSorted[0] < -38 and monthlyTemperatureSorted[8] < 10):
            return result + 'd'
        elif (monthlyTemperatureSorted[11] >= 22):
            return result + 'a'
        # at least 4 month temperature >= 10 Celsius
        elif (monthlyTemperatureSorted[8] >= 10):
            return result + 'b'
        else:
            return result + 'c'
    return 'undefined'



In [38]:
import pandas as pd

data = pd.read_csv('concat-filter.csv')

data.interpolate(method='linear', inplace=True)

data_dict = {}

for index, row in data.iterrows():
    station = row['station']
    year = row['year']
    if 1977 <= year <= 2020:
        if (station, year) not in data_dict:
            data_dict[(station, year)] = {'tavg': [], 'prcp': []}

        data_dict[(station, year)]['tavg'].append(row['tavg'])
        data_dict[(station, year)]['prcp'].append(row['prcp'])
    
data_dict

{('17096',
  1977): {'tavg': [-12.0,
   -3.1,
   0.2,
   6.6,
   10.7,
   15.0,
   18.7,
   19.9,
   16.0,
   5.0,
   3.8,
   -7.1], 'prcp': [12.0,
   18.0,
   30.0,
   51.0,
   75.0,
   69.0,
   12.0,
   32.0,
   9.0,
   33.0,
   13.0,
   21.0]},
 ('17096',
  1978): {'tavg': [-7.3,
   -3.1,
   -0.2,
   4.0,
   10.4,
   13.7,
   21.1,
   18.6,
   15.9,
   9.6,
   -1.0,
   -2.2], 'prcp': [41.0,
   24.0,
   35.0,
   77.0,
   70.0,
   39.0,
   11.0,
   3.0,
   25.0,
   41.0,
   9.0,
   25.0]},
 ('17096',
  1979): {'tavg': [-4.1,
   -2.2,
   0.6,
   5.9,
   10.9,
   14.2,
   17.9,
   21.4,
   17.0,
   7.5,
   3.3,
   -5.8], 'prcp': [14.0,
   50.0,
   19.0,
   93.0,
   94.0,
   72.0,
   39.0,
   7.0,
   1.0,
   102.0,
   81.0,
   21.0]},
 ('17096',
  1980): {'tavg': [-10.5,
   -8.1,
   -2.5,
   5.0,
   10.7,
   16.7,
   22.6,
   20.0,
   14.7,
   7.7,
   3.8,
   -2.5], 'prcp': [53.0,
   30.0,
   36.0,
   32.0,
   55.0,
   3.0,
   7.0,
   21.0,
   24.0,
   46.0,
   37.0,
   30.0]},
 ('17096'

In [39]:
# TODO：没处理N/S；只保留1977-2020; 有的站（1990，10）被（1990，1）匹配
result_data = []

for (station, year), data in data_dict.items():
    temperature = data['tavg']
    precipitation = data['prcp']
    climate_type = onehot[koppen(temperature, precipitation, 'N')]

    print(station, year)
    result_data.append({'station': station, 'year': year, 'type': climate_type})

result_df = pd.DataFrame(result_data, columns=['station', 'year', 'type'])

result_df.to_csv('classification.csv', index=False)

17096 1977
17096 1978
17096 1979
17096 1980
17096 1981
17096 1982
17096 1983
17096 1984
17096 1985
17096 1986
17096 1987
17096 1988
17096 1989
17096 1990
17096 1991
17096 1992
17096 1993
17096 1994
17096 1995
17096 1996
17096 1997
17096 1998
17096 1999
17096 2000
17096 2001
17096 2002
17096 2003
17096 2004
17096 2005
17096 2006
17096 2007
17096 2008
17096 2009
17096 2010
17096 2011
17096 2012
17096 2013
17096 2014
17096 2015
17096 2016
17096 2017
17096 2018
17096 2019
17096 2020
36064 1977
36064 1978
36064 1979
36064 1980
36064 1981
36064 1982
36064 1983
36064 1984
36064 1985
36064 1986
36064 1987
36064 1988
36064 1989
36064 1990
36064 1991
36064 1992
36064 1993
36064 1994
36064 1995
36064 1996
36064 1997
36064 1998
36064 1999
36064 2000
36064 2001
36064 2002
36064 2003
36064 2004
36064 2005
36064 2006
36064 2007
36064 2008
36064 2009
36064 2010
36064 2011
36064 2012
36064 2013
36064 2014
36064 2015
36064 2016
36064 2017
36064 2018
36064 2019
36064 2020
08175 1977
08175 1978
08175 1979

72259 2011
72259 2012
72259 2013
72259 2014
72259 2015
72259 2016
72259 2017
72259 2018
72259 2019
72259 2020
10379 1977
10379 1978
10379 1979
10379 1980
10379 1981
10379 1982
10379 1983
10379 1984
10379 1985
10379 1986
10379 1987
10379 1988
10379 1989
10379 1990
10379 1991
10379 1992
10379 1993
10379 1994
10379 1995
10379 1996
10379 1997
10379 1998
10379 1999
10379 2000
10379 2001
10379 2002
10379 2003
10379 2004
10379 2005
10379 2006
10379 2007
10379 2008
10379 2009
10379 2010
10379 2011
10379 2012
10379 2013
10379 2014
10379 2015
10379 2016
10379 2017
10379 2018
10379 2019
10379 2020
34740 1977
34740 1978
34740 1979
34740 1980
34740 1981
34740 1982
34740 1983
34740 1984
34740 1985
34740 1986
34740 1987
34740 1988
34740 1989
34740 1990
34740 1991
34740 1992
34740 1993
34740 1994
34740 1995
34740 1996
34740 1997
34740 1998
34740 1999
34740 2000
34740 2001
34740 2002
34740 2003
34740 2004
34740 2005
34740 2006
34740 2007
34740 2008
34740 2009
34740 2010
34740 2011
34740 2012
34740 2013

## LSTM
0.1607

In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
scaler = MinMaxScaler()
data['type'] = scaler.fit_transform(data['type'].values.reshape(-1, 1))
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练LSTM模型
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(1, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]
y_test = y_test.reshape(-1, 1)

y_pred = model.predict(X_test)

# 6. 计算整体的RMSE
rmse = mean_squared_error(y_test, y_pred, squared = False)

# 打印整体的RMSE
print("Overall RMSE:", rmse)

Epoch 1/100
1001/1001 [==============================] - 2s 1ms/step - loss: 0.0544
Epoch 2/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0272
Epoch 3/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0272
Epoch 4/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0272
Epoch 5/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0272
Epoch 6/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0272
Epoch 7/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0271
Epoch 8/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0271
Epoch 9/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0271
Epoch 10/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0271
Epoch 11/100
1001/1001 [==============================] - 1s 1ms/step - loss: 0.0271
Epoch 12/100
1001/1001 [==============================] - 1s 1ms/step - lo

In [13]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练LSTM模型
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(1, 1)))
model.add(Dense(units=32, activation='softmax'))  # 32分类问题，使用softmax激活函数
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]

y_pred = model.predict(X_test)

# 6. 计算整体的RMSE（对于分类问题不是必需的）
# rmse = mean_squared_error(y_test, y_pred, squared=False)

# 7. 计算F1分数（多分类问题）
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')

# 打印整体的F1分数
print("Overall F1 Score:", f1)

Epoch 1/100
1001/1001 [==============================] - 4s 2ms/step - loss: 2.5321 - accuracy: 0.2590
Epoch 2/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.3067 - accuracy: 0.3047
Epoch 3/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2559 - accuracy: 0.3214
Epoch 4/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2054 - accuracy: 0.3322
Epoch 5/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.1686 - accuracy: 0.3350
Epoch 6/100
1001/1001 [==============================] - 2s 2ms/step - loss: 2.1496 - accuracy: 0.3381
Epoch 7/100
1001/1001 [==============================] - 2s 2ms/step - loss: 2.1392 - accuracy: 0.3367
Epoch 8/100
1001/1001 [==============================] - 2s 2ms/step - loss: 2.1306 - accuracy: 0.3367
Epoch 9/100
1001/1001 [==============================] - 2s 2ms/step - loss: 2.1260 - accuracy: 0.3354
Epoch 10/100
1001/1001 [==============================] - 1s 1ms/step - l

## ARIMA
0.3209

In [50]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
scaler = MinMaxScaler()
data['type'] = scaler.fit_transform(data['type'].values.reshape(-1, 1))
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

train_series = train_data['type']
test_series = test_data['type']

# 4. 拟合ARIMA模型
p, d, q = 40, 1, 1  # 根据您的时间序列分析结果选择适当的阶数
arima_model = ARIMA(train_series, order=(p, d, q))
arima_result = arima_model.fit()

# 5. 计算预测
forecast = arima_result.predict(start=len(train_series), end=len(train_series) + len(test_series) - 1, typ='levels')

# 6. 计算整体的RMSE
rmse = np.sqrt(mean_squared_error(test_series, forecast))

# 打印整体的RMSE
print("Overall RMSE:", rmse)

/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this

Overall RMSE: 0.3209209217434175


## CNN
0.2900

F1 = 0.2945

In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
scaler = MinMaxScaler()
data['type'] = scaler.fit_transform(data['type'].values.reshape(-1, 1))

# 3. 分离数据为训练集和测试集
train_data = data[data['year'] < 2020]  # 使用年份小于2020的数据作为训练集
test_data = data[data['year'] == 2020]  # 使用年份等于2020的数据作为测试集

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练CNN模型
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=1, activation='relu', input_shape=(1, 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]
y_test = y_test.reshape(-1, 1)

y_pred = model.predict(X_test)

# 6. 计算整体的RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# 打印整体的RMSE
print("Overall RMSE:", rmse)

29/29 [==============================] - 0s 715us/step
Overall RMSE: 0.29001519311294865


In [12]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练CNN模型
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(units=32, activation='softmax'))  # 32分类问题，使用softmax激活函数
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]

y_pred = model.predict(X_test)

# 6. 计算F1分数（多分类问题）
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')

# 打印整体的F1分数
print("Overall F1 Score:", f1)


Epoch 1/100
1001/1001 [==============================] - 2s 862us/step - loss: 2.7385 - accuracy: 0.2179
Epoch 2/100
1001/1001 [==============================] - 1s 876us/step - loss: 2.4868 - accuracy: 0.2682
Epoch 3/100
1001/1001 [==============================] - 1s 849us/step - loss: 2.3658 - accuracy: 0.2943
Epoch 4/100
1001/1001 [==============================] - 1s 888us/step - loss: 2.3065 - accuracy: 0.3096
Epoch 5/100
1001/1001 [==============================] - 1s 907us/step - loss: 2.2692 - accuracy: 0.3270
Epoch 6/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2452 - accuracy: 0.3301
Epoch 7/100
1001/1001 [==============================] - 1s 927us/step - loss: 2.2268 - accuracy: 0.3348
Epoch 8/100
1001/1001 [==============================] - 1s 892us/step - loss: 2.2137 - accuracy: 0.3377
Epoch 9/100
1001/1001 [==============================] - 1s 832us/step - loss: 2.2046 - accuracy: 0.3390
Epoch 10/100
1001/1001 [==============================] -

Epoch 79/100
1001/1001 [==============================] - 1s 870us/step - loss: 2.0429 - accuracy: 0.4030
Epoch 80/100
1001/1001 [==============================] - 1s 899us/step - loss: 2.0430 - accuracy: 0.4016
Epoch 81/100
1001/1001 [==============================] - 1s 974us/step - loss: 2.0402 - accuracy: 0.4053
Epoch 82/100
1001/1001 [==============================] - 1s 883us/step - loss: 2.0430 - accuracy: 0.4016
Epoch 83/100
1001/1001 [==============================] - 1s 916us/step - loss: 2.0392 - accuracy: 0.4037
Epoch 84/100
1001/1001 [==============================] - 1s 879us/step - loss: 2.0403 - accuracy: 0.4041
Epoch 85/100
1001/1001 [==============================] - 1s 892us/step - loss: 2.0399 - accuracy: 0.4036
Epoch 86/100
1001/1001 [==============================] - 1s 985us/step - loss: 2.0396 - accuracy: 0.4024
Epoch 87/100
1001/1001 [==============================] - 1s 983us/step - loss: 2.0377 - accuracy: 0.4031
Epoch 88/100
1001/1001 [======================

## 多层感知机MLP
0.2896

F1 = 

In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
scaler = MinMaxScaler()
data['type'] = scaler.fit_transform(data['type'].values.reshape(-1, 1))

# 3. 分离数据为训练集和测试集
train_data = data[data['year'] < 2020]  # 使用年份小于2020的数据作为训练集
test_data = data[data['year'] == 2020]  # 使用年份等于2020的数据作为测试集

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练MLP模型
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(1,)))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]
y_test = y_test.reshape(-1, 1)

y_pred = model.predict(X_test)

# 6. 计算整体的RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# 打印整体的RMSE
print("Overall RMSE:", rmse)

29/29 [==============================] - 0s 829us/step
Overall RMSE: 0.28965927641797423


In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练MLP模型
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=1))
model.add(Dense(units=32, activation='softmax'))  # 32分类问题，使用softmax激活函数
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]

y_pred = model.predict(X_test)

# 6. 计算F1分数（多分类问题）
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')

# 打印整体的F1分数
print("Overall F1 Score:", f1)

Epoch 1/100
1001/1001 [==============================] - 1s 925us/step - loss: 2.7963 - accuracy: 0.2133
Epoch 2/100
1001/1001 [==============================] - 1s 840us/step - loss: 2.4837 - accuracy: 0.2696
Epoch 3/100
1001/1001 [==============================] - 1s 810us/step - loss: 2.3719 - accuracy: 0.2952
Epoch 4/100
1001/1001 [==============================] - 2s 2ms/step - loss: 2.3196 - accuracy: 0.3103
Epoch 5/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2884 - accuracy: 0.3261
Epoch 6/100
1001/1001 [==============================] - 1s 914us/step - loss: 2.2674 - accuracy: 0.3334
Epoch 7/100
1001/1001 [==============================] - 1s 890us/step - loss: 2.2515 - accuracy: 0.3346
Epoch 8/100
1001/1001 [==============================] - 1s 871us/step - loss: 2.2385 - accuracy: 0.3381
Epoch 9/100
1001/1001 [==============================] - 1s 882us/step - loss: 2.2309 - accuracy: 0.3380
Epoch 10/100
1001/1001 [==============================] - 1

## 线性回归LR

In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
scaler = MinMaxScaler()
data['type'] = scaler.fit_transform(data['type'].values.reshape(-1, 1))

# 3. 分离数据为训练集和测试集
train_data = data[data['year'] < 2020]  # 使用年份小于2020的数据作为训练集
test_data = data[data['year'] == 2020]  # 使用年份等于2020的数据作为测试集

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练线性回归模型
model = LinearRegression()
model.fit(X_train, y_train)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]
y_test = y_test.reshape(-1, 1)

y_pred = model.predict(X_test)

# 6. 计算整体的RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# 打印整体的RMSE
print("Overall RMSE:", rmse)

Overall RMSE: 0.28830355094116983


## SVM

In [89]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data[['type']]  # 注意 X_train 是 DataFrame
y_train = train_data['type']

X_test = test_data[['type']]  # 注意 X_test 是 DataFrame
y_test = test_data['type']

# 4. 创建和训练 SVM 模型
model = SVC(kernel='linear', C=0.0001)  # 使用线性核，可以根据需要更改核函数和参数
model.fit(X_train, y_train)

# 5. 预测2020年的类型
y_pred = model.predict(X_test)

# 6. 计算 F1 分数（多分类问题）
f1 = f1_score(y_test, y_pred, average='micro')

# 打印整体的 F1 分数
print("Overall F1 Score:", f1)

Overall F1 Score: 0.32704795204795206


## 朴素贝叶斯

In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

# 4. 合并训练和测试数据以进行一致的特征处理
combined_data = pd.concat([train_data, test_data])

# 5. 使用独热编码处理 "station" 列
combined_data = pd.get_dummies(combined_data, columns=['station'], drop_first=True)

# 6. 恢复分离的训练和测试数据
train_data = combined_data.iloc[:len(train_data), :]
test_data = combined_data.iloc[len(train_data):, :]

X_train = train_data.drop(['type'], axis=1)
y_train = train_data['type']

X_test = test_data.drop(['type'], axis=1)
y_test = test_data['type']

# 7. 创建和训练朴素贝叶斯模型
model = MultinomialNB()
model.fit(X_train, y_train)

# 8. 预测2020年的类型
y_pred = model.predict(X_test)

# 9. 计算F1分数（多分类问题）
f1 = f1_score(y_test, y_pred, average='micro')

# 打印整体的F1分数
print("Overall F1 Score:", f1)


Overall F1 Score: 0.09290709290709293


## RNN

In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1, 1)  # 添加时间步信息
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练SimpleRNN模型
model = Sequential()
model.add(SimpleRNN(units=50, activation='relu', input_shape=(1, 1)))
model.add(Dense(units=32, activation='softmax'))  # 32分类问题，使用softmax激活函数
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1, 1)  # 添加时间步信息
y_test = test_data['type'].values[1:]

y_pred = model.predict(X_test)

# 6. 计算整体的RMSE（对于分类问题不是必需的）
# rmse = mean_squared_error(y_test, y_pred, squared=False)

# 7. 计算F1分数（多分类问题）
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='micro')

# 打印整体的F1分数
print("Overall F1 Score:", f1)

Epoch 1/100
1001/1001 [==============================] - 2s 1ms/step - loss: 2.7302 - accuracy: 0.2219
Epoch 2/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.4740 - accuracy: 0.2695
Epoch 3/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.3643 - accuracy: 0.2991
Epoch 4/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.3122 - accuracy: 0.3111
Epoch 5/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2792 - accuracy: 0.3247
Epoch 6/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2549 - accuracy: 0.3331
Epoch 7/100
1001/1001 [==============================] - 1s 999us/step - loss: 2.2369 - accuracy: 0.3354
Epoch 8/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2244 - accuracy: 0.3396
Epoch 9/100
1001/1001 [==============================] - 1s 1ms/step - loss: 2.2152 - accuracy: 0.3410
Epoch 10/100
1001/1001 [==============================] - 1s 1ms/step -

ValueError: y_true and y_pred have different number of output (1!=32)

## DNN

In [10]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# 1. 从CSV文件加载数据
data = pd.read_csv('classification.csv')

# 2. 数据准备
data = data.sort_values(by=['station', 'year'])

# 3. 分离数据为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

X_train = train_data['type'].values[:-1]
y_train = train_data['type'].values[1:]

X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

# 4. 创建和训练深度神经网络模型
model = Sequential()
model.add(Dense(units=128, activation='relu', input_shape=(1, 1)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='softmax'))  # 32分类问题，使用softmax激活函数
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=1)

# 5. 预测2020年的类型
X_test = test_data['type'].values[:-1]
X_test = X_test.reshape(-1, 1)
y_test = test_data['type'].values[1:]

y_pred = model.predict(X_test)

# 6. 计算F1分数（多分类问题）
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='micro')

# 打印整体的F1分数
print("Overall F1 Score:", f1)

251/251 [==============================] - 0s 642us/step


ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets